In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip install rouge_score
!pip install nltk
!pip install bert_score
!pip install peft
!pip install --upgrade bitsandbytes
!pip install optuna
!pip install -U datasets

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=13ae8fecd9f2514e103a8e0537ff37193823b26117edaade57b5b43a27160256
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 831.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import optuna
import numpy as np

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TrainingArguments, Trainer, DataCollatorForSeq2Seq, BitsAndBytesConfig, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score
from peft import get_peft_model, LoraConfig
import pandas as pd
import transformers
import torch
import random
from datasets import Dataset


from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from bert_score import score as bert_score

# Load Model

In [ ]:
model_id = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model_id)


# Load the model with 4-bit quantization using bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16,  # Mixed precision for compute (can use float16 or bfloat16)
    bnb_4bit_use_double_quant=True,       # Enable double quantization for more memory savings
    bnb_4bit_quant_type="nf4"             # Use NormalFloat4 (nf4) quantization, recommended for QLoRA
)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=bnb_config)

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

# Load & Prepare *Data*

In [ ]:
# Load the evaluation dataset
df = pd.read_csv('/content/medquad_summarized.csv')

In [ ]:
df['input_text'] = df['question']
df['output_text'] = df['summary']
# Convert DataFrame to Hugging Face Dataset format
dataset = Dataset.from_pandas(df[['input_text', 'output_text']])

# Load Tokenizer

In [ ]:
# Ensure the tokenizer has a pad_token set
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token = tokenizer.unk_token  # Use unknown token as pad token instead of eos

# Make sure the pad_token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS as padding if not set


# Resize model embeddings to match the tokenizer's vocabulary size
model.resize_token_embeddings(len(tokenizer))



def tokenize_function(examples):
    return tokenizer(
        examples['input_text'],
        text_target=examples['output_text'],
        padding="max_length",      # Max_length for consistent
        truncation=True,
        max_length=256,         # Adjust as per your VRAM or reduce if needed
    )

# Apply the tokenization to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/14979 [00:00<?, ? examples/s]

In [ ]:
# Split the dataset into training (80%) and validation (20%) sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Use only a small subset of the data for Hyperparameter
train_subset = train_dataset.select(range(4000))
eval_subset = eval_dataset.select(range(2000))

# Hyperparameter Tuning

In [ ]:
def objective(trial):
    # Suggest hyperparameters for LoraConfig
    lora_rank = trial.suggest_int('lora_r', 16, 26)  # Adjust the range as necessary
    lora_alpha = trial.suggest_int('lora_alpha', 16, 24)
    lora_dropout = trial.suggest_float('lora_dropout', 0.1, 0.15)

    # Define LoRA configuration
    lora_config = LoraConfig(
        r=lora_rank,                         # LoRA rank
        lora_alpha=lora_alpha,               # Scaling factor
        lora_dropout=lora_dropout,           # Dropout to prevent overfitting
        target_modules=["query_key_value", "dense"],  # Adjust based on your model architecture
        bias="none",                         # Keep as "none"
        task_type="CAUSAL_LM",              # Task type
    )

    # # Define LoRA configuration
    # lora_config = LoraConfig(
    #     r=16,                         # LoRA rank
    #     lora_alpha=32,               # Scaling factor
    #     lora_dropout=0.5,           # Dropout to prevent overfitting
    #     target_modules=["query_key_value","dense"],  # Adjust based on your model architecture
    #     bias="none",                         # Keep as "none"
    #     task_type="CAUSAL_LM",              # Task type
    # )


    model.enable_input_require_grads()
    # Apply LoRA to the model
    model_with_lora = get_peft_model(model, lora_config)
    model_with_lora.print_trainable_parameters()



    # Suggest hyperparameters for training
    learning_rate = trial.suggest_float('learning_rate', 5e-6, 1e-5, log=True)
    num_train_epochs = trial.suggest_int('num_train_epochs', 3, 5)
    per_device_train_batch_size = 4
    # # Define training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps = max(1, 8 // per_device_train_batch_size),  # Adjust to maintain effective batch size
        learning_rate=learning_rate,
        num_train_epochs=num_train_epochs,
        eval_strategy="steps",            # Evaluate once per 200 steps to monitor progress
        eval_steps=200,                    # Less frequent evaluation
        save_steps=400,                     # Adjusted to be a multiple of eval_steps
        logging_strategy="steps",
        logging_steps = 200,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        bf16=True,
    )

    # training_args = TrainingArguments(
    #     output_dir="./results",
    #     per_device_train_batch_size=4,
    #     gradient_accumulation_steps = 4,  # Adjust to maintain effective batch size
    #     learning_rate=1e-6,
    #     num_train_epochs=2,
    #     eval_strategy="steps",            # Evaluate once per 200 steps to monitor progress
    #     eval_steps=10,                    # Less frequent evaluation
    #     logging_strategy="steps",
    #     logging_steps = 10,
    #     load_best_model_at_end=True,
    #     metric_for_best_model="eval_loss",
    #     greater_is_better=False,
    #     bf16=True,
    # )

    # Adjust model configuration for VRAM efficiency
    model_with_lora.config.use_cache = False  # Disable caching to reduce memory usage
    model_with_lora.gradient_checkpointing_enable()  # Enable gradient checkpointing

    # # Decoding hyperparameters
    # top_k = trial.suggest_int('top_k', 20, 50)
    # top_p = trial.suggest_uniform('top_p', 0.7, 0.95)
    # temperature = trial.suggest_uniform('temperature', 0.5, 1.0)
    # repetition_penalty = trial.suggest_uniform('repetition_penalty', 1.1, 1.5)
    # max_length = trial.suggest_int('max_length', 128, 256)

    # # Update the generation configuration
    # model_with_lora.config.update({
    #     "do_sample": True,
    #     "top_k": top_k,
    #     "top_p": top_p,
    #     "temperature": temperature,
    #     "repetition_penalty": repetition_penalty,
    #     "max_length": max_length
    # })

    # Initialize the data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_with_lora, padding=True)


    # Initialize Trainer
    trainer = Trainer(
        model=model_with_lora,
        args=training_args,
        train_dataset=train_subset,
        eval_dataset=eval_subset,
        data_collator=data_collator
    )

    # for name, param in model_with_lora.named_parameters():
    #     if param.dtype in [torch.float16, torch.float32, torch.float64]:
    #         param.requires_grad = True

    # def check_for_nan(module, grad_input, grad_output):
    #   for gi in grad_input:
    #       if gi is not None and torch.isnan(gi).any():
    #           print(f"NaN detected in gradients for {module}")

    #     #  Register hooks to check for NaNs
    # for module in model_with_lora.modules():
    #     module.register_backward_hook(check_for_nan)

    # Set the trainer to explicitly return the loss
    trainer.can_return_loss = True
    # Train and evaluate
    trainer.train()
    eval_results = trainer.evaluate(eval_dataset=eval_dataset)

    return eval_results['eval_loss']


In [ ]:
# Create the study and optimize
study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=20)  # Adjust `n_trials` to the number of hyperparameter combinations you want to try

# Display the best trial
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2024-10-01 11:42:59,420] A new study created in memory with name: no-name-dc751884-f1e5-4579-a6ca-6323abc3117b


trainable params: 14,057,472 || all params: 6,935,778,176 || trainable%: 0.2027


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
200,8.717100,6.375253
400,5.244100,4.835126
600,4.724200,4.607574
800,4.511600,4.412804
1000,4.330300,4.275606
1200,4.233700,4.191270
1400,4.153700,4.145138
1600,4.129200,4.121818
1800,4.101100,4.109850
2000,4.101700,4.106264


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-pac

...

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
200,7.698500,5.088428
400,4.748900,4.537871
600,4.378500,4.225451
800,4.141500,4.090102
1000,4.044100,4.030007
1200,4.008600,3.992758
1400,3.958200,3.967000
1600,3.950100,3.951042
1800,3.929600,3.936923
2000,3.919200,3.929631


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-pac

[I 2024-10-02 08:43:25,722] Trial 19 finished with value: 3.914379119873047 and parameters: {'lora_r': 24, 'lora_alpha': 20, 'lora_dropout': 0.12310057100368679, 'learning_rate': 8.893630366717144e-06, 'num_train_epochs': 5}. Best is trial 19 with value: 3.914379119873047.


Best trial:
  Value: 3.914379119873047
  Params: 
    lora_r: 24
    lora_alpha: 20
    lora_dropout: 0.12310057100368679
    learning_rate: 8.893630366717144e-06
    num_train_epochs: 5
